In [70]:
import pandas 
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import sigmoid_kernel 

In [71]:
song_df = pandas.read_csv("song_data.csv")


In [72]:
song_df.head()

,song_id,title,release,artist_name,year
0,SOQMMHC12AB0180CB8,Silent Night,Monster Ballads X-Mas,Faster Pussy cat,2003
1,SOVFVAK12A8C1350D9,Tanssi vaan,Karkuteillä,Karkkiautomaatti,1995
2,SOGTUKN12AB017F4F1,No One Could Ever,Butter,Hudson Mohawke,2006
3,SOBNYVR12A8C13558C,Si Vos Querés,De Culo,Yerba Brava,2003
4,SOHSBXH12A8C13B0DF,Tangle Of Aspens,Rene Ablaze Presents Winter Sessions,Der Mystic,0


In [73]:
song_df.dropna(inplace=True)

In [74]:
song_df = song_df[["title","artist_name"]]

In [75]:
song_df_x,song_df_y = train_test_split(song_df,test_size=0.99,random_state=121)

In [76]:
song_df_x.head()

,title,artist_name
553102,Expecting,Minnie Riperton
8380,It Serves Me Right To Suffer,John Lee Hooker
6564,Overture,Laruso
930248,Mix Up,Chaka Demus
595225,Closer Than Close,The Stylistics


In [77]:
song_df_x.reset_index(drop=True,inplace=True)

In [78]:
song_df_x['title'] = song_df_x['title'].str.lower() 

C:\Users\SUJEET\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [79]:
tfidf = TfidfVectorizer(stop_words="english")

In [80]:
tfv_matrix = tfidf.fit_transform(song_df_x['artist_name'])

In [82]:
sig = sigmoid_kernel(tfv_matrix,tfv_matrix) 

In [83]:
indices = pandas.Series(song_df_x.index, index=song_df_x['title']).drop_duplicates()

In [88]:
import pickle

with open("song_sig.pickle","wb") as f:
    pickle.dump(sig,f)

with open("song_indices.pickle","wb") as f:
    pickle.dump(indices,f)

with open("song_title.pickle","wb") as f:
    pickle.dump(song_df_x,f)



In [84]:
indices.head()

title
expecting                       0
it serves me right to suffer    1
overture                        2
mix up                          3
closer than close               4
dtype: int64

In [85]:
def song_recommend(title):
    title = title.lower()
    idx = indices[title]
    sig_score = list(enumerate(sig[idx]))
    sig_score = sorted(sig_score, key=lambda x:x[1], reverse = True)
    sig_score = sig_score[1:11]
    song_indices = [i[0] for i in sig_score]
    
    return song_df_x['title'].iloc[song_indices]
    

In [89]:
song_recommend('expecting')

145                                   les fleurs
9205    inside my love (digitally remastered 93)
1                   it serves me right to suffer
2                                       overture
3                                         mix up
4                              closer than close
5                                          slime
6                                      star song
7                               a starting point
8                         dancing on the ceiling
Name: title, dtype: object